In [18]:
import numpy as np
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

In [19]:
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

In [20]:
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


데이터 전처리

In [21]:
# 중복여부 확인
train_df.document.nunique()

146182

In [22]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146183, 3)

In [23]:
# null데이터 있는지 확인
train_df.isna().sum()

id          0
document    1
label       0
dtype: int64

In [24]:
# Null 데이터 제거
train_df=train_df.dropna(how='any')
train_df.shape

(146182, 3)

In [25]:
# 부정, 긍정 데이터 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트데이터 셋

In [26]:
# 중복여부 확인
test_df.document.nunique()

49157

In [27]:
# 중복 데이터 제거
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49158, 3)

In [28]:
# null데이터 있는지 확인
test_df.isna().sum()

id          0
document    1
label       0
dtype: int64

In [29]:
# Null 데이터 제거
test_df=test_df.dropna(how='any')
test_df.shape

(49157, 3)

2. 텍스트 전처리
- train dataset

In [31]:
# 한글과 공백 이외의 문자는 제거
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


# 데이터가 다 지워져서 ''가 된 데이터 --> np.nan으로 변경한 후 제거

In [32]:
train_df.document.replace('',np.nan, inplace=True)
train_df.isna().sum()

id            0
document    391
label         0
dtype: int64

In [33]:
train_df = train_df.dropna(how='any')
train_df.shape

(145791, 3)

In [34]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
test_df.document.replace('',np.nan, inplace=True)
test_df = test_df.dropna(how='any')
test_df.shape

(48995, 3)

- 전처리가 끝난 데이터는 저장

In [35]:
train_df.to_csv('naver_movie_train.tsv', sep='\t',index=False)
test_df.to_csv('naver_movie_test.tsv', sep='\t',index=False)

# 한글처리

In [36]:
from konlpy.tag import Okt
okt = Okt()

In [37]:
text = '홈포스터보고 초딩영화줄오버연기조차 가볍지 않구나'
okt.morphs(text)

['홈', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍지', '않구나']

In [39]:
okt.morphs(text, stem=True) # stem은 원형을 추출

['홈', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']

In [40]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text, stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [41]:
stop_text = '은 는 이 가 을 를 의 에게 에 들 좀 잘 과 도 으로 자 와 ㅋㅋ ㅠㅠ ㅎㅎ'
stopwords = stop_text.split()


In [45]:
from tqdm.notebook import tqdm
X_train =[]
for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_list = [word for word in morphs if word not in stopwords]
    tmp_str = ' '.join(tmp_list)
    X_train.append(tmp_str)

  0%|          | 0/145791 [00:00<?, ?it/s]

In [46]:
from tqdm.notebook import tqdm
X_test =[]
for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence,stem=True)
    tmp_str = ' '.join([word for word in morphs if word not in stopwords])
    X_test.append(tmp_str)

  0%|          | 0/48995 [00:00<?, ?it/s]

In [47]:
y_train = train_df.label.values
y_test = test_df.label.values

#### 4. feature 변환

In [48]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer()
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)

#### 5. 모델 학습

In [49]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_cv, y_train)
lr.score(X_test_cv, y_test)

0.8251862434942341

#### 6. 실제 테스트

In [51]:
review1 = 'ㅈㄴ 잼있음 꼭 봐라 눈물주의'
review2 = '저 정말 스파이더맨 팬입니다. 정말 역겨움을 참느라 힘들었습니다. 이 영화가 재미있다고 하는 사람들 이해할 수가 없네요. 과거 스파이더맨이 와서 이미 팬들이 알고 있는 내용을 아주 천천히 설명하는 장면이 영화의 반을...'


In [53]:
# 전처리
import re
review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review1)
review2 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review2)
review2

'저 정말 스파이더맨 팬입니다 정말 역겨움을 참느라 힘들었습니다 이 영화가 재미있다고 하는 사람들 이해할 수가 없네요 과거 스파이더맨이 와서 이미 팬들이 알고 있는 내용을 아주 천천히 설명하는 장면이 영화의 반을'

In [55]:
# 불용어 처리
morphs = okt.morphs(review1, stem=True)
review1 = ' '.join(word for word in morphs if word not in stopwords)
review1

'ㅈㄴ 재다 꼭 보다 눈물 주의'

In [57]:
review2 = ' '.join(word for word in okt.morphs(review2, stem=True) if word not in stopwords)
review2

'저 정말 스파이더맨 팬 이다 정말 역겹다 움 참다 힘들다 영화 재미있다 하다 사람 이해 하다 수가 없다 과거 스파이더맨 오다 이미 팬 알 고 있다 내용 아주 천천히 설명 하다 장면 영화 반'

In [58]:
# Feature 변환
review1_cv = cvect.transform([review1])

In [59]:
lr.predict(review1_cv)

array([1], dtype=int64)

In [60]:
review2_cv = cvect.transform([review2])
lr.predict(review2_cv)

array([1], dtype=int64)